In [4]:
import numpy as np
from scipy.sparse import csr_matrix, lil_matrix
import networkx as nx
import matplotlib.pyplot as plt


# Energy Function

In [ ]:
def graph_coloring_energy(adj_matrix, coloring, A=1):
    """
    Computes the energy function for a given graph coloring.

    Parameters:
    - adj_matrix (numpy.ndarray): Adjacency matrix of the graph (NxN).
    - coloring (numpy.ndarray): Coloring matrix (NxC), where C is the number of colors.
    - A (float): Penalty coefficient.

    Returns:
    - float: Energy value of the given coloring.
    """
    N, C = coloring.shape  # Number of vertices (N) and colors (C)

    # Term 1: Vertex Constraint - Each vertex should have exactly one color
    vertex_constraint = sum((1 - np.sum(coloring[i, :]))**2 for i in range(N))

    # Term 2: Edge Constraint - No two adjacent vertices should share the same color
    edge_constraint = sum(
        np.sum(coloring[u, :] * coloring[v, :])  # Sum over colors where both vertices have the same color
        for u in range(N) for v in range(N) if adj_matrix[u, v] == 1
    )

    # Compute total energy
    # print('\nvertex constraint')
    # print(vertex_constraint)
    # print('\nedge constraint')
    # print(edge_constraint)
    energy = A * (vertex_constraint + edge_constraint)
    
    return energy

def compute_local_field(adj_matrix, q, A=1):
    """
    Compute the local field for DSB algorithm.
    
    Parameters:
    - adj_matrix (numpy.ndarray): Adjacency matrix of the graph
    - q (numpy.ndarray): Current binary state variables
    - A (float): Penalty coefficient
    
    Returns:
    - numpy.ndarray: Local field for each variable
    """
    N, C = q.shape
    
    # Initialize field
    h = np.zeros_like(q)
    
    # Vertex constraint contribution
    for i in range(N):
        sum_colors = np.sum(q[i, :])
        for c in range(C):
            h[i, c] += -2 * A * (1 - sum_colors)
    
    # Edge constraint contribution
    for u in range(N):
        for v in range(N):
            if adj_matrix[u, v] == 1:
                for c in range(C):
                    h[u, c] += A * q[v, c]
    
    return h

def discrete_simulated_bifurcation(adj_matrix, num_colors, max_iter=1000, dt=0.1, A=1, alpha_init=0.5, alpha_scale=0.998):
    """
    Optimize graph coloring using Discrete Simulated Bifurcation.
    
    Parameters:
    - adj_matrix (numpy.ndarray): Adjacency matrix of the graph
    - num_colors (int): Number of colors to use
    - max_iter (int): Maximum number of iterations
    - dt (float): Time step size
    - A (float): Penalty coefficient
    - alpha_init (float): Initial value of alpha
    - alpha_scale (float): Scaling factor for alpha each iteration
    
    Returns:
    - numpy.ndarray: Optimized coloring
    - list: Energy history
    """
    N = len(adj_matrix)  # Number of vertices
    
    # Initialize binary variables and their velocities
    q = np.random.choice([-1, 1], size=(N, num_colors))
    p = np.zeros((N, num_colors))
    
    # Initialize alpha (bifurcation parameter)
    alpha = alpha_init
    
    # Initialize energy history
    energy_history = []
    
    # Convert q to coloring (0/1 matrix)
    def q_to_coloring(q_matrix):
        coloring = np.zeros_like(q_matrix)
        coloring[q_matrix > 0] = 1
        return coloring
    
    # Main loop
    for iteration in range(max_iter):
        # Convert q to coloring format for energy calculation
        coloring = q_to_coloring(q)
        
        # Calculate current energy
        energy = graph_coloring_energy(adj_matrix, coloring, A)
        energy_history.append(energy)
        
        # Calculate local field
        h = compute_local_field(adj_matrix, coloring, A)
        
        # Update momentum (p)
        p = p - dt * h
        
        # Update position (q)
        for i in range(N):
            for c in range(num_colors):
                # Update using discrete simulated bifurcation equations
                if p[i, c] > 0:
                    q[i, c] = 1
                elif p[i, c] < 0:
                    q[i, c] = -1
                # If p[i, c] is exactly 0, q[i, c] remains unchanged
        
        # Decrease alpha (annealing)
        alpha *= alpha_scale
        
        # Check if solution is valid (termination condition)
        if energy == 0:
            break
    
    # Return the best coloring found
    final_coloring = q_to_coloring(q)
    return final_coloring, energy_history

def run_dsb_for_best_result(adj_matrix, num_colors, R = 100, max_iter=20000, dt=0.05, A=1.0, alpha_init=0.5, alpha_scale=0.999):
    best_coloring = None
    lowest_final_energy = float('inf')
    
    for _ in range(R):
        coloring, energy_history = discrete_simulated_bifurcation(
            adj_matrix, 
            num_colors, 
            max_iter=max_iter, 
            dt=dt, 
            A=A, 
            alpha_init=alpha_init,
            alpha_scale=alpha_scale
        )
        
        if energy_history[-1] < lowest_final_energy:
            lowest_final_energy = energy_history[-1]
            best_coloring = coloring
    
    return best_coloring, energy_history

if __name__ == 'main':

    adj_matrix = np.array([[0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

    num_colors = 2

    coloring, energy_history = run_dsb_for_best_result(adj_matrix, num_colors, R = 100, max_iter=20000, dt=0.05, A=1.0, alpha_init=0.5, alpha_scale=0.999)